In [ ]:
!pip install -qqq --upgrade bitsandbytes transformers peft accelerate datasets trl flash_attn
!pip install huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 83.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 

In [ ]:
!pip install wandb -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from random import randrange

import torch
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)
from trl import SFTTrainer

In [ ]:
# 'hf_adapter_repo' is a variable that holds the repository name for the Hugging Face model adapter.

# 'edumunozsala/phi-3-mini-QLoRA' is the repository name, where 'edumunozsala' is the username of the repository owner and 'phi-3-mini-QLoRA' is the name of the model adapter.

# 'model_name, hf_adapter_repo, compute_dtype' is a line of code that returns the values of the 'model_name', 'hf_adapter_repo', and 'compute_dtype' variables.

# This block of code is used to set the repository name for the Hugging Face model adapter and then return the values of the 'model_name', 'hf_adapter_repo', and 'compute_dtype' variables.
hf_adapter_repo = "dhanishetty/phi-3-mini-QLoRA"
model_name = "microsoft/Phi-3-mini-4k-instruct"
compute_dtype = "torch.bfloat16"

model_name, hf_adapter_repo, compute_dtype

('microsoft/Phi-3-mini-4k-instruct',
 'dhanishetty/phi-3-mini-QLoRA',
 'torch.bfloat16')

In [ ]:
# 'peft_model_id' and 'tr_model_id' are variables that hold the identifiers for the PEFT model and the transformer model, respectively.

# 'AutoModelForCausalLM.from_pretrained(tr_model_id, trust_remote_code=True, torch_dtype=compute_dtype)' is a function that loads a pre-trained transformer model for causal language modeling. 'tr_model_id' is the identifier for the pre-trained model, 'trust_remote_code=True' allows the execution of code from the model file, and 'torch_dtype=compute_dtype' sets the data type for the PyTorch tensors.

# 'PeftModel.from_pretrained(model, peft_model_id)' is a function that loads a pre-trained PEFT model. 'model' is the transformer model and 'peft_model_id' is the identifier for the pre-trained PEFT model.

# 'model.merge_and_unload()' is a method that merges the PEFT model with the transformer model and then unloads the PEFT model.

# This block of code is used to load a pre-trained transformer model and a pre-trained PEFT model, merge the two models, and then unload the PEFT model.
peft_model_id = hf_adapter_repo
tr_model_id = model_name

model = AutoModelForCausalLM.from_pretrained(tr_model_id, trust_remote_code=True, torch_dtype="auto")
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

In [ ]:
# 'tokenizer' is a variable that holds the tokenizer.

# 'AutoTokenizer.from_pretrained(peft_model_id)' is a function from the Hugging Face Transformers library that loads a pre-trained tokenizer. 'peft_model_id' is the identifier for the pre-trained tokenizer.

# This line of code is used to load a pre-trained tokenizer.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

tokenizer_config.json:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

# 'hf_model_repo' is a variable that holds the repository name for the Hugging Face model.

# This line of code is used to reference the repository name for the Hugging Face model.
hf_model_repo = "dhanishetty/phi-3-mini-QLoRA-merged"

In [ ]:
# 'merged_model_id' is a variable that holds the identifier for the merged model.

# 'hf_model_repo' is the repository name for the Hugging Face model.

# 'model.push_to_hub(merged_model_id)' is a method that pushes the merged model to the Hugging Face Model Hub. 'merged_model_id' is the identifier for the merged model.

# 'tokenizer.push_to_hub(merged_model_id)' is a method that pushes the tokenizer to the Hugging Face Model Hub. 'merged_model_id' is the identifier for the tokenizer.

# This block of code is used to save the merged model and the tokenizer to the Hugging Face Model Hub.
# SAve the model merged to the Hub
merged_model_id = hf_model_repo
model.push_to_hub(merged_model_id)
tokenizer.push_to_hub(merged_model_id)

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dhanishetty/phi-3-mini-QLoRA-merged/commit/c84c4d6d14ac19673d547261c34f595bb3bc19d6', commit_message='Upload tokenizer', commit_description='', oid='c84c4d6d14ac19673d547261c34f595bb3bc19d6', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
device_map = {"": 0}

device_map, compute_dtype

({'': 0}, 'torch.bfloat16')

In [ ]:
# This block of code is used to import the necessary libraries, set the seed for reproducibility, and load a pre-trained tokenizer and model.

# 'import torch' is a line of code that imports the PyTorch library, which is a popular open-source machine learning library.

# 'from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed' is a line of code that imports the 'AutoTokenizer', 'AutoModelForCausalLM', and 'set_seed' functions from the Hugging Face Transformers library.

# 'set_seed(1234)' is a line of code that sets the seed for the random number generator to '1234'. This is done to ensure that the results are reproducible.

# 'tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)' is a line of code that loads a pre-trained tokenizer from the Hugging Face Model Hub. 'hf_model_repo' is the repository name for the model and 'trust_remote_code=True' allows the execution of code from the model file.

# 'model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype=compute_dtype, device_map=device_map)' is a line of code that loads a pre-trained model for causal language modeling from the Hugging Face Model Hub. 'hf_model_repo' is the repository name for the model, 'trust_remote_code=True' allows the execution of code from the model file, 'torch_dtype=compute_dtype' sets the data type for the PyTorch tensors, and 'device_map=device_map' sets the device mapping.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(1234)  # For reproducibility

tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype="auto", device_map=device_map) # compute "auto" dev_map "cuda"


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
# This block of code is used to prepare the dataset for training.

# 'dataset_chatml = dataset.map(create_message_column)' is a line of code that applies the 'create_message_column' function to each element in the 'dataset'. The result is stored in 'dataset_chatml'.

# 'dataset_chatml = dataset_chatml.map(format_dataset_chatml)' is a line of code that applies the 'format_dataset_chatml' function to each element in 'dataset_chatml'.

# 'dataset_chatml = dataset_chatml.train_test_split(test_size=0.05)' is a line of code that splits 'dataset_chatml' into a training set and a test set. The test set size is 5% of the total dataset.

# 'dataset_chatml' is a line of code that returns the 'dataset_chatml' variable.

# This block of code is used to create a message column in the dataset, format the dataset, and then split the dataset into a training set and a test set.
## prepare the dataset

dataset_name = "openai/gsm8k"
dataset_split = 'main'

dataset = load_dataset(dataset_name, dataset_split)

def create_message_column(row):
    messages = []
    user = {
        "content": f" Input: {row['question']}",
        "role": "user"
    }
    messages.append(user)
    assistant = {
        "content": f"{row['answer']}",
        "role": "assistant"
    }
    messages.append(assistant)
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

dataset_chatml = dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)
dataset_chatml = dataset_chatml['test']
dataset_chatml

Dataset({
    features: ['question', 'answer', 'messages', 'text'],
    num_rows: 1319
})

In [ ]:
# 'pipe' is a variable that holds the pipeline for text generation.

# 'pipeline("text-generation", model=model, tokenizer=tokenizer)' is a function from the Hugging Face Transformers library that creates a pipeline for text generation. 'model' is the pre-trained model and 'tokenizer' is the pre-trained tokenizer.

# This line of code is used to create a pipeline for text generation using the pre-trained model and tokenizer.
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# 'pipe' is an instance of the 'pipeline' class from the 'transformers' library. It is used to generate responses to prompts.

# 'pipe.tokenizer' is the tokenizer of the 'pipe' pipeline. It is used to convert text into a format that the model can understand.

# 'apply_chat_template' is a function of the tokenizer that formats a chat message in a way that the model can understand.

# '[{"role": "user", "content": dataset_chatml['test'][0]['messages'][0]['content']}]' is a list of dictionaries that represents a chat message. Each dictionary contains a 'role' field that indicates the role of the sender (in this case, 'user') and a 'content' field that contains the text of the message.

# 'tokenize=False' is a parameter that controls whether the 'apply_chat_template' function should tokenize the input. In this case, it is set to 'False', which means that the function will not tokenize the input.

# 'add_generation_prompt=True' is a parameter that controls whether the 'apply_chat_template' function should add a generation prompt to the input. In this case, it is set to 'True', which means that the function will add a generation prompt.

# This line of code tests the 'apply_chat_template' function by applying it to the first message in the test set of the 'dataset_chatml' dataset.
pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml[0]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)


"<|user|>\n Input: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|end|>\n<|assistant|>\n"

In [ ]:

# 'test_inference' is a function that generates a response to a given prompt.

# 'prompt' is the input parameter, which is expected to be a string that contains the user's message.

# 'prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)' applies a chat template to the 'prompt' and assigns the result back to 'prompt'. The 'apply_chat_template' function is part of the tokenizer of the 'pipe' pipeline. It formats the 'prompt' in a way that the model can understand.

# 'outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95, max_time= 180)' generates a response to the 'prompt' using the 'pipe' pipeline. The parameters control the generation process. For example, 'max_new_tokens=256' limits the length of the generated response to 256 tokens, and 'temperature=0.3' controls the randomness of the generation process (lower values make the output more deterministic).

# 'return outputs[0]['generated_text'][len(prompt):].strip()' returns the generated response. The '[len(prompt):]' part removes the original 'prompt' from the beginning of the 'generated_text', and the 'strip()' part removes leading and trailing whitespace.
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95, max_time= 180)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
test_inference(dataset_chatml[0]['messages'][0]['content'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


"Janet's ducks lay 16 eggs per day.\nShe eats 3 eggs for breakfast every morning, so she has 16 - 3 = 13 eggs left.\nShe bakes muffins with 4 eggs, so she has 13 - 4 = 9 eggs left.\nShe sells the remaining 9 eggs at the farmers' market for $2 per egg, so she makes 9 * $2 = $<<9*2=18>>18 at the farmers' market every day."

In [ ]:

# 'datasets' is a library that provides access to a large collection of datasets and evaluation metrics for Natural Language Processing (NLP).

# 'load_metric' is a function from the 'datasets' library that loads an evaluation metric.

# 'from datasets import load_metric' imports the 'load_metric' function from the 'datasets' library. This allows you to use 'load_metric' directly instead of having to write 'datasets.load_metric'.
from datasets import load_metric

In [ ]:
!pip install rouge_score
# 'load_metric' is a function from the 'datasets' library that loads an evaluation metric.

# 'rouge_metric = load_metric("rouge", trust_remote_code=True)' loads the ROUGE metric and assigns it to the variable 'rouge_metric'.

# The 'trust_remote_code' parameter is set to 'True', which means that the function will trust and execute remote code. This is necessary because the ROUGE metric is not included in the 'datasets' library by default and needs to be downloaded from the Hugging Face Model Hub.
rouge_metric = load_metric("rouge", trust_remote_code=True)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=6e9e3e494ad192b142c8e898a9e05d87ca9bfb3f654bc665b11320be8d305a2a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:

# 'calculate_rogue' is a function that calculates the ROUGE score for a given row of data.

# 'row' is the input parameter, which is expected to be a dictionary-like object that contains the fields 'messages' and 'output'.

# 'response = test_inference(row['messages'][0]['content'])' generates a response by running the 'test_inference' function on the 'content' field of the first message in the 'messages' field of 'row'.

# 'result = rouge_metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)' computes the ROUGE score by comparing the generated 'response' with the 'output' field of 'row'. The 'use_stemmer' parameter is set to 'True', which means that the ROUGE metric will use a stemmer to reduce words to their root form before comparison.

# 'result = {key: value.mid.fmeasure * 100 for key, value in result.items()}' transforms the 'result' dictionary by multiplying each value by 100. This is done to convert the ROUGE scores from a range of 0-1 to a range of 0-100.

# 'result['response']=response' adds the generated 'response' to the 'result' dictionary.

# 'return result' returns the 'result' dictionary, which now contains the ROUGE scores and the generated response.
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = rouge_metric.compute(predictions=[response], references=[row['answer']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [ ]:
metricas = dataset_chatml.select(range(0,500)).map(calculate_rogue, batched=False)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:

# 'numpy' is a library in Python that provides support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.

# 'import numpy as np' imports the 'numpy' library and gives it the alias 'np'. This means that you can use 'np' instead of 'numpy' to refer to the library in the rest of your code.
import numpy as np

In [ ]:
# 'np.mean' is a function from the 'numpy' library that calculates the arithmetic mean of an array.

# 'metricas' is a dictionary that contains the evaluation metrics of the model.

# 'metricas['rouge1']', 'metricas['rouge2']', 'metricas['rougeL']', and 'metricas['rougeLsum']' are lists of ROUGE scores for each evaluated sample.

# ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate automatic summarization and machine translation. It compares the overlap of n-grams between the system's output and the reference summaries.

# 'print("Rouge 1 Mean: ",np.mean(metricas['rouge1']))' prints the mean ROUGE-1 score.

# 'print("Rouge 2 Mean: ",np.mean(metricas['rouge2']))' prints the mean ROUGE-2 score.

# 'print("Rouge L Mean: ",np.mean(metricas['rougeL']))' prints the mean ROUGE-L score.

# 'print("Rouge Lsum Mean: ",np.mean(metricas['rougeLsum']))' prints the mean ROUGE-Lsum score.
print("Rouge 1 Mean: ",np.mean(metricas['rouge1']))
print("Rouge 2 Mean: ",np.mean(metricas['rouge2']))
print("Rouge L Mean: ",np.mean(metricas['rougeL']))
print("Rouge Lsum Mean: ",np.mean(metricas['rougeLsum']))

Rouge 1 Mean:  61.6651360303827
Rouge 2 Mean:  37.40708644328783
Rouge L Mean:  49.97512761268223
Rouge Lsum Mean:  57.621618574625806


In [ ]:

# 'dataset_chatml' is a dictionary-like object that contains the dataset.

# 'dataset_chatml['test']' retrieves the test set from the 'dataset_chatml' dataset.

# 'dataset_chatml['test'][0]' retrieves the first sample from the test set.

# 'dataset_chatml['test'][0]['output']' retrieves the 'output' field from the first sample in the test set. This 'output' field typically contains the expected response or output for the given sample.
dataset_chatml[0]['answer']

'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'

In [ ]:
# 'num_samples' is a variable that specifies the number of samples to be used for the evaluation.

# In this case, 'num_samples=500' means that 500 samples will be used for the evaluation.
num_samples=50

In [ ]:
%%time
# '%%time' is a magic command in Jupyter Notebook that measures the execution time of the cell it is placed in.

# 'prompts' is a list of prompts that are generated by applying a chat template to the first message of each sample in the test set of the 'dataset_chatml' dataset.

# 'pipe' is an instance of the 'pipeline' class from the 'transformers' library. It is used to generate responses to the prompts.

# 'outputs' is a list of responses that are generated by the 'pipe' pipeline.

# 'preds' is a list of predictions that are extracted from the 'outputs'. Each prediction is the text that follows the "<|assistant|>\n" tag in the generated response.

# 'references' is a list of reference outputs that are extracted from the test set of the 'dataset_chatml' dataset.

# 'rouge_metric.add_batch(predictions=preds, references=references)' adds a batch of predictions and references to the 'rouge_metric' for evaluation.
prompts = [pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml[i]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)
                                              for i in range(num_samples)]
outputs = pipe(prompts, batch_size=4, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95,
                   max_time= 180)
preds = [outputs[i][0]['generated_text'].split("<|assistant|>\n")[1].strip() for i in range(len(outputs))]
references= [dataset_chatml[i]['answer'] for i in range(len(outputs))]
rouge_metric.add_batch(predictions=preds, references=references)

CPU times: user 2min 5s, sys: 172 ms, total: 2min 5s
Wall time: 2min 5s


In [ ]:

# 'rouge_metric' is an instance of the Rouge metric class from the 'datasets' library.

# 'compute' is a method of the Rouge metric class that calculates the Rouge scores.

# 'use_stemmer=True' is an argument that indicates whether to use stemming when calculating the Rouge scores. Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form.

# So, 'result = rouge_metric.compute(use_stemmer=True)' calculates the Rouge scores with stemming and stores the result in the 'result' variable.
result = rouge_metric.compute(use_stemmer=True)

In [ ]:

# 'np.mean' is a function from the 'numpy' library that calculates the arithmetic mean of an array.

# 'result' is a dictionary that contains the evaluation metrics of the model.

# 'result['rouge1']', 'result['rouge2']', 'result['rougeL']', and 'result['rougeLsum']' are lists of ROUGE scores for each evaluated sample.

# ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate automatic summarization and machine translation. It compares the overlap of n-grams between the system's output and the reference summaries.

# 'print("Rouge 1 Mean: ",np.mean(result['rouge1']))' prints the mean ROUGE-1 score.

# 'print("Rouge 2 Mean: ",np.mean(result['rouge2']))' prints the mean ROUGE-2 score.

# 'print("Rouge L Mean: ",np.mean(result['rougeL']))' prints the mean ROUGE-L score.

# 'print("Rouge Lsum Mean: ",np.mean(result['rougeLsum']))' prints the mean ROUGE-Lsum score.
print("Rouge 1 Mean: ",np.mean(result['rouge1']))
print("Rouge 2 Mean: ",np.mean(result['rouge2']))
print("Rouge L Mean: ",np.mean(result['rougeL']))
print("Rouge Lsum Mean: ",np.mean(result['rougeLsum']))

Rouge 1 Mean:  0.6251033497678627
Rouge 2 Mean:  0.3735674576038987
Rouge L Mean:  0.48896006761286454
Rouge Lsum Mean:  0.5882014297604081


In [ ]:

# 'result' is a dictionary that contains the evaluation metrics of the model.

# 'result['rouge1']' retrieves the value of the ROUGE-1 score from the 'result' dictionary.

# ROUGE-1 is a metric for evaluating automatic summarization of texts and machine translation. It compares the overlap of unigrams (single words) between the system's output and the reference summaries.
result['rouge1']

AggregateScore(low=Score(precision=0.5635074049011431, recall=0.6091349554898254, fmeasure=0.5695366110020739), mid=Score(precision=0.6093509168855822, recall=0.6603009133805359, fmeasure=0.6073534707660191), high=Score(precision=0.6570383795315964, recall=0.7086335502699489, fmeasure=0.6410739456840401))